In [4]:
import spacy
from spacy.tokens import DocBin
from spacy.training import Example
from spacy.util import Config
from spacy.training import Corpus
import random
from tqdm import tqdm
import pandas as pd 

In [6]:
nlp_fr = spacy.load("fr_core_news_md")
print("spacy model charged success")

spacy model charged success


In [8]:
texte_simple = '''Je voudrais aller de Ermont à Sannois. ...'''
doc = nlp_fr(texte_simple)
spacy.displacy.render(doc, style="ent", jupyter=True)

In [10]:
# charger le fichier CSV
df = pd.read_csv('../../dataset/raw/initial_training_data.csv')
print("\nFicheir csv chargé\n")


Ficheir csv chargé



In [20]:
# premieres lignes du dataset pour verifier le chargement
print(df.head())

                                            sentence  entity_start  \
0  Quels sites touristiques visiter en allant de ...            46   
1  Quels sites touristiques visiter en allant de ...            54   
2  Combien de temps faut-il pour aller de PARIS à...            39   
3  Combien de temps faut-il pour aller de PARIS à...            47   
4  Quels sont les moyens de transport les plus éc...            70   

   entity_end  entity_type  
0          51    DEPARTURE  
1          63  DESTINATION  
2          44    DEPARTURE  
3          51  DESTINATION  
4          75    DEPARTURE  


In [22]:
# on cree un conteneur pour stocker les documents au format spaCy
db = DocBin()
print("\nconteneur crée\n")


conteneur crée



In [24]:
# on melange les  pour éviter tout biais dans l'apprentissage
df = df.sample(frac=1).reset_index(drop=True)

In [26]:
# on convertit les donnees au format spaCy
for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    text = row['sentence']
    doc = nlp_fr.make_doc(text)  # on cree un document spaCy a partir du texte
    ents = []
    
    # recuperer les infos des labels
    start = row['entity_start']
    end = row['entity_end']
    label = row['entity_type']
    
    # creation d'une entite spaCy à partir des coordonnees de debut et de fin
    span = doc.char_span(start, end, label=label, alignment_mode="contract")
    if span is not None:
        ents.append(span)
    
    doc.ents = ents  # assigner les entites au document
    db.add(doc)  # ajouter le document au DocBin

100%|██████████| 499000/499000 [05:41<00:00, 1460.26it/s]


In [28]:
# enregistre les donnes converties au format spaCy
db.to_disk("../../dataset/processed/training_data.spacy")

In [30]:
# ajout du composant NER au modele actuel
ner = nlp_fr.get_pipe("ner")

In [32]:
# ajoute de nouveaux labels d'entites a partir des donnes
for _, row in df.iterrows():
    ner.add_label(row['entity_type'])


In [33]:
print("Les nouvelles étiquettes NER ont été ajoutées au modèle.")


Les nouvelles étiquettes NER ont été ajoutées au modèle.
